#### Implement the baseline algos here

In [1]:
import sys; sys.path.insert(0, "../"); from utils import *
import pickle

%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from pde_diff import *
import robust_PDE_FIND
from robust_PDE_FIND import TrainSTRidge, compute_err, print_pde, RobustPCA, Robust_LRSTR
import scipy.io as sio

Running Python 3.9.7
You can use npar for np.array
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['diff', 'save']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
# Loading the KS sols
# data = sio.loadmat("../R_DLGA_datasets/KS.mat")
data = sio.loadmat("../PDE_FIND_experimental_datasets/kuramoto_sivishinky.mat")

t = data['tt'][0,:]
x = data['x'][:,0]

idx_x = np.arange(len(x))
idx_x = idx_x[idx_x%2==0]
idx_t = np.arange(len(t))
idx_t = idx_t[idx_t%2==0]

x = x[idx_x]
t = t[idx_t]

time_dims = t.shape[0]
spatial_dims = x.shape[0]

total_points = time_dims*spatial_dims
print("Total data points:", total_points)

X, T = np.meshgrid(x, t)
Exact = data['uu']
u = Exact[idx_x][:, idx_t]

# DATA_PATH = "../deephpms_data/KS_new3.pkl"
# data = pickle_load(DATA_PATH)
# t = data['t']
# x = data['x']
# X, T = np.meshgrid(x, t)
# Exact = data['u']
# u = Exact

# from rkstiff import grids
# from rkstiff import if34

# # Computing the KS sol
# # uniform grid spacing, real-valued u -> construct_x_kx_rfft
# N = 256
# # a, b = 0, 16*np.pi
# M = 2
# a, b = -M*np.pi, M*np.pi
# # a, b = -M, M
# x, kx = grids.construct_x_kx_rfft(N, a, b)

# L = kx**2*(1-kx**2)
# def NL(uFFT):
#     u = np.fft.irfft(uFFT)
#     ux = np.fft.irfft(1j*kx*uFFT)
#     return -np.fft.rfft(u*ux)

# u0 = -np.sin(np.pi*x/10)
# # u0 = np.cos(x/16)*(1.+np.sin(x/16))
# u0FFT = np.fft.rfft(u0)
# solver = if34.IF34(linop=L,NLfunc=NL)
# ufFFT = solver.evolve(u0FFT, t0=0, tf=10) # store every 20th step in solver.u and solver.t

# U = []
# for uFFT in solver.u:
#     U.append(np.fft.irfft(uFFT))
# U = np.array(U)
# t = np.array(solver.t)

# X, T = np.meshgrid(x, t)
# Exact = U.T
# u = Exact

# spatial_dims = Exact.shape[0]
# time_dims = Exact.shape[1]

Total data points: 64512


In [3]:
# fig1 = figure()
# ax = fig1.gca(projection='3d')
# surf = ax.plot_surface(X, T, u.T, rstride=1, cstride=1, cmap=cm.coolwarm,
#     linewidth=0, antialiased=False)
# title('Kuramoto Sivashinsky Equation', fontsize = 20)
# xlabel('x', fontsize = 16)
# ylabel('t', fontsize = 16)

#### STR

In [4]:
dt = t[1]-t[0]
dx = x[2]-x[1]
Ut, R, rhs_des = build_linear_system(u, dt, dx, D=5, P=5, time_diff = 'FD', space_diff = 'FD')

In [5]:
idx = np.random.choice(R.shape[0], R.shape[0], replace=False)

In [6]:
w = TrainSTRidge(R,Ut,10**-6,50)
# w = TrainSTRidge(R[idx, :],Ut[idx, :],10**-6,50)
print("PDE derived using STRidge")
print_pde(w, rhs_des)

/Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_small_KS2/robust_PDE_FIND.py:845: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
/Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_small_KS2/robust_PDE_FIND.py:890: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
/Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_small_KS2/robu

PDE derived using STRidge
u_t = (-0.887105 +0.000000i)uu_{x}
    + (-0.914812 +0.000000i)u_{xx}
    + (-0.933346 +0.000000i)u_{xxxx}
   


In [7]:
print(u.shape)
total_points = u.shape[0]*u.shape[1]
print(total_points)

(512, 126)
64512


In [8]:
# 1024*89 -> sample 256*89 to train

In [11]:
with open("../deephpms_data/KS_new3.pkl", "wb") as f:
    pickle.dump(dict({'x':x, 't':t, 'u':u}), f)

In [11]:
errs = 100*(np.abs(npar([-0.997619, -1.048322, -1.055897])+1))
errs.mean(), errs.std()

(3.5533333333333346, 2.364533865738074)

#### STR + noise

In [12]:
noise_intensity = 1e-7
numpy.random.seed(0)
un = u + noise_intensity*std(u)*np.random.randn(u.shape[0],u.shape[1])

In [13]:
Utn, Rn, rhs_des = build_linear_system(un, dt, dx, D=5, P=5, time_diff = 'FD', space_diff = 'FD')

In [14]:
# Solve with STRidge
w = TrainSTRidge(Rn,Utn,10**-5, 50)
print("PDE derived using STRidge")
print_pde(w, rhs_des)

PDE derived using STRidge
u_t = (-0.999528 +0.000000i)uu_{x}
    + (-1.001889 +0.000000i)u_{xx}
    + (-1.006649 +0.000000i)u_{xxxx}
   


In [15]:
errs = 100*(npar([-0.783763, -0.812064, -0.807380])+1)
errs.mean(), errs.std()

(19.8931, 1.2385695808741097)

#### DLrSR

In [14]:
u_in = u

Z, E1 = RobustPCA(u_in, lam_2 = 1)

diff_type = "fd"

if diff_type == "poly":
    ZUtn, ZRn, rhs_des = build_linear_system(Z, dt, dx, D=5, P=5, 
                                     time_diff = 'poly', space_diff = 'poly', 
                                     width_x = 20, width_t = 20, deg_x = 6)
elif diff_type == "fd":
    ZUtn, ZRn, rhs_des = build_linear_system(Z, dt, dx, D=5, P=5, time_diff = 'FD', space_diff = 'FD')

iteration:1, err:1067.415154544554, nc_norm:374.40273112357954 eta1:0.009617105529935862
iteration:2, err:1.5776804968715487e-12, nc_norm:9462.345220077881 eta1:0.01057881608292945


In [15]:
ZUtn.shape

(22784, 1)

In [16]:
if diff_type == "poly":
    double_width = 0
    for n in range(0, 101):
        if ZUtn.shape[0] == (spatial_dims-n)*(time_dims-n):
            double_width = n
            nx = spatial_dims-n
            nt = time_dims-n
            print(double_width); break
elif diff_type == "fd":
    (nx,nt) = spatial_dims, time_dims

In [17]:
# Solve with DLrSR
Utn1 = np.reshape(ZUtn, (nx,nt))
w, X, E2 = Robust_LRSTR(ZRn, Utn1, rhs_des, lam_1 = 1e-4, lam_3 = 0.15, lam_4 = 1, d_tol = 5)
print_pde(w, rhs_des)

iteration:1, err:309.0862924559844, nc_norm:227.27950554574994 eta2:0.03558481379411077
u_t = 
u_t = (-0.053144 +0.000000i)uu_{x}
   
u_t = (-0.781462 +0.000000i)uu_{x}
    + (-0.800460 +0.000000i)u_{xx}
    + (-0.805789 +0.000000i)u_{xxxx}
   
u_t = (-1.202778 +0.000000i)uu_{x}
    + (-1.266484 +0.000000i)u_{xx}
    + (-1.274580 +0.000000i)u_{xxxx}
   
u_t = (-1.231895 +0.000000i)uu_{x}
    + (-1.302967 +0.000000i)u_{xx}
    + (-1.311633 +0.000000i)u_{xxxx}
   
u_t = (-1.086341 +0.000000i)uu_{x}
    + (-1.146310 +0.000000i)u_{xx}
    + (-1.154452 +0.000000i)u_{xxxx}
   
u_t = (-0.981328 +0.000000i)uu_{x}
    + (-1.031867 +0.000000i)u_{xx}
    + (-1.039472 +0.000000i)u_{xxxx}
   
u_t = (-0.965909 +0.000000i)uu_{x}
    + (-1.014408 +0.000000i)u_{xx}
    + (-1.021805 +0.000000i)u_{xxxx}
   
u_t = (-0.986086 +0.000000i)uu_{x}
    + (-1.035959 +0.000000i)u_{xx}
    + (-1.043342 +0.000000i)u_{xxxx}
   
u_t = (-0.998330 +0.000000i)uu_{x}
    + (-1.049225 +0.000000i)u_{xx}
    + (-1.056643 +0

In [18]:
errs = np.abs(100*(npar([-0.996344, -1.047156, -1.054584])+1))
errs.mean(), errs.std()

(3.5131999999999977, 2.2462527729532122)

#### DLrSR + noise (Normal distributed noise)

In [19]:
u_in = un
Z, E1 = RobustPCA(u_in, lam_2 = 1)
ZUtn, ZRn, rhs_des = build_linear_system(Z, dt, dx, D=5, P=5, time_diff = 'FD', space_diff = 'FD')

iteration:1, err:1067.4120956604484, nc_norm:374.4662645613448 eta1:0.009617105823414697
iteration:2, err:2.023393753421382e-12, nc_norm:9462.38023402353 eta1:0.010578816405756168


In [20]:
# Solve with DLrSR
Utn1 = np.reshape(ZUtn, (nx,nt))
w, X, E2=Robust_LRSTR(ZRn, Utn1, rhs_des, lam_1 = 1e-4, lam_3 = 0.15, lam_4 = 1, d_tol = 5)
print_pde(w, rhs_des)

iteration:1, err:309.0825598842347, nc_norm:227.39352810368345 eta2:0.035584614135765705
u_t = 
u_t = (-0.053181 +0.000000i)uu_{x}
   
u_t = (-0.619896 +0.000000i)uu_{x}
    + (-0.621807 +0.000000i)u_{xx}
    + (-0.617886 +0.000000i)u_{xxxx}
   
u_t = (0.170612 +0.000000i)u^3
    + (-0.032934 +0.000000i)u^5
    + (-1.562760 +0.000000i)uu_{x}
    + (0.032372 +0.000000i)u^5u_{x}
    + (-0.828919 +0.000000i)u_{xx}
    + (0.365723 +0.000000i)u^2u_{xx}
    + (-0.476496 +0.000000i)uu_{xxx}
    + (-0.874345 +0.000000i)u_{xxxx}
    + (0.040933 +0.000000i)u^4u_{xxxx}
   
u_t = (-0.962636 +0.000000i)uu_{x}
    + (-1.003189 +0.000000i)u_{xx}
    + (-0.996915 +0.000000i)u_{xxxx}
   
u_t = (-0.846964 +0.000000i)uu_{x}
    + (-0.884143 +0.000000i)u_{xx}
    + (-0.878044 +0.000000i)u_{xxxx}
   
u_t = (0.204283 +0.000000i)u^3
    + (-0.044902 +0.000000i)u^5
    + (-1.552602 +0.000000i)uu_{x}
    + (-0.389894 +0.000000i)u^2u_{x}
    + (-0.177390 +0.000000i)u^3u_{x}
    + (0.108886 +0.000000i)u^4u_{x}
 

In [23]:
errs = np.abs(100*(npar([-0.784982, -0.813629, -0.808917])+1))
errs.mean(), errs.std()

(19.749066666666668, 1.254209701587243)

#### Notes on the AAAI'20 paper
    - The algo needs grid. Not possible to run the algo on freely distributed data points.
    - Assume strictly sparse-distributed noise.
    - It is not clear how to tune the lagrange multiplier hyper parameters so that the optimal true PDE is obtained.
    - The PDE solution and the pde diff features are not learned or updated joinly, hence, the sub-optimal equation is discovered, especially in the noisy case.